In [6]:
from Simulations import overall

ImportError: cannot import name 'overall' from 'Simulations' (c:\Users\gorse\OneDrive\Documents\ensai\troisieme\reinforcement_L\travail_projet_Vauto\RL-final-project\Simulations.py)

In [5]:
R=overall()

NameError: name 'overall' is not defined